In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.stats import zscore
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# --- 1. Load Data ---
file_path = 'final_dataset.parquet'

if not os.path.exists(file_path):
	display(f"Error: The file '{file_path}' was not found. Please ensure it's in the correct directory.")
	exit() # Exit if file not found
else:
	try:
		df = pd.read_parquet(file_path)
		display(f"Successfully loaded '{file_path}'. Shape: {df.shape}")
		print(f"Number of rows (lines): {df.shape[0]}")
		display("First 5 rows of the dataset:")
		display(df.head())
	except Exception as e:
		display(f"Error loading Parquet file: {e}")
		exit() # Exit if loading fails

# --- 2. Feature Selection ---
features = [
	'rating',
	'text_len',
	'rating_deviation',
	'sentiment_polarity',
	'sentiment_subjectivity',
	'excessive_exclaim',
	'avg_rating',
	'num_of_reviews',
	'log_num_reviews',
	# 'latitude',
	# 'longitude',
	'price_encoded',
	'year',
	'month',
	'weekday',
	'hour',
	'cat_American restaurant',
	'cat_Coffee shop',
	'cat_Department store',
	'cat_Fast food restaurant',
	'cat_Grocery store',
	'cat_Hotel',
	'cat_Mexican restaurant',
	'cat_Other',
	'cat_Pizza restaurant',
	'cat_Restaurant',
	'cat_Shopping mall'
]

# Check if all required features exist in the DataFrame
missing_features = [f for f in features if f not in df.columns]
if missing_features:
	display(f"Error: Missing required features in the dataset: {missing_features}")
	display("Please ensure your 'final_dataset.parquet' contains these columns.")
	features = [f for f in features if f in df.columns] # Proceed with available
	if not features:
		display("No valid features remaining. Exiting.")
		exit()
	else:
		display(f"Proceeding with available features: {features}")

data = df[features].copy()

# Handle potential NaNs in features (e.g., fill with mean or median)
for col in features:
	if data[col].isnull().any():
		data[col] = data[col].fillna(data[col].mean())
		display(f"Filled NaN values in '{col}' with its mean.")

# --- 3. Data Preprocessing (Scaling) ---
scaler = MinMaxScaler()
scaled_data_np = scaler.fit_transform(data) # Keep as numpy for now

# Convert to PyTorch tensors
scaled_data_tensor = torch.tensor(scaled_data_np, dtype=torch.float32)

# Split data into training and validation sets
X_train_tensor, X_val_tensor = train_test_split(scaled_data_tensor, test_size=0.2, random_state=42)

# Create DataLoader for batching
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, X_train_tensor) # Input and target are the same for AE
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, X_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# --- 4. Autoencoder Model Definition (PyTorch) ---
class Autoencoder(nn.Module):
	def __init__(self, input_dim, encoding_dim):
		super(Autoencoder, self).__init__()
		# Encoder
		self.encoder = nn.Sequential(
			nn.Linear(input_dim, encoding_dim * 2),
			nn.LeakyReLU(0.01),  # <--- Change to LeakyReLU
			nn.Linear(encoding_dim * 2, encoding_dim),
			nn.LeakyReLU(0.01)   # <--- Change to LeakyReLU
		)
		# Decoder (it's good practice to change it here too)
		self.decoder = nn.Sequential(
			nn.Linear(encoding_dim, encoding_dim * 2),
			nn.LeakyReLU(0.01),  # <--- Change to LeakyReLU
			nn.Linear(encoding_dim * 2, input_dim),
			nn.Sigmoid()
		)
		
	def forward(self, x):
		encoded = self.encoder(x)
		decoded = self.decoder(encoded)
		return decoded

input_dim = X_train_tensor.shape[1]
encoding_dim = 10 # Dimensionality of the latent space (can be tuned)

model = Autoencoder(input_dim, encoding_dim)
display("\nAutoencoder Model Summary (PyTorch):")
display(model) # PyTorch displays module structure directly

# Define Loss function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 5. Training the Autoencoder ---
epochs = 50
display(f"\nTraining Autoencoder for {epochs} epochs...")

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
display(f"Using device: {device}")

for epoch in range(epochs):
	model.train() # Set model to training mode
	running_loss = 0.0
	for data_batch, _ in train_loader: # _ for target, which is same as input
		data_batch = data_batch.to(device)

		optimizer.zero_grad() # Clear gradients
		outputs = model(data_batch) # Forward pass
		loss = criterion(outputs, data_batch) # Calculate loss
		loss.backward() # Backward pass
		optimizer.step() # Update weights

		running_loss += loss.item() * data_batch.size(0)

	train_loss = running_loss / len(train_dataset)

	# Validation phase
	model.eval() # Set model to evaluation mode
	val_loss = 0.0
	with torch.no_grad(): # No gradient calculation needed
		for data_batch_val, _ in val_loader:
			data_batch_val = data_batch_val.to(device)
			outputs_val = model(data_batch_val)
			loss_val = criterion(outputs_val, data_batch_val)
			val_loss += loss_val.item() * data_batch_val.size(0)
	val_loss = val_loss / len(val_dataset)

	display(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

display("\nTraining complete.")

# --- 6. Calculate Reconstruction Error (PyTorch) ---
model.eval() # Set model to evaluation mode
all_reconstructions = []
with torch.no_grad():
	for i in range(0, len(scaled_data_tensor), batch_size):
		batch = scaled_data_tensor[i:i + batch_size].to(device)
		reconstructed_batch = model(batch)
		all_reconstructions.append(reconstructed_batch.cpu().numpy())

reconstructions_np = np.vstack(all_reconstructions)

# Calculate Mean Squared Error (MSE) between original and reconstructed data
reconstruction_errors = np.mean(np.square(scaled_data_np - reconstructions_np), axis=1)

# --- 7. Z-scoring the Reconstruction Error ---
z_scored_reconstruction_errors = zscore(reconstruction_errors)

# Add the z-scored errors to your original DataFrame
df['ae_reconstruction_error_zscore'] = z_scored_reconstruction_errors

display("\nReconstruction errors calculated and z-scored.")
display("First 10 reviews with their z-scored autoencoder reconstruction error:")
display(df[['ae_reconstruction_error_zscore'] + features].head(10))

# Identify potential anomalies
threshold = 2.0
anomalies = df[df['ae_reconstruction_error_zscore'] > threshold]
display(f"\nNumber of potential anomalies (z-score > {threshold}): {len(anomalies)}")
# --- MODIFIED PART ---
# Display more anomalies and all of their original features
if not anomalies.empty:
	print("\nDisplaying the top 20 anomalies with all their features (sorted by anomaly score):")

	# Set pandas display options to see all data without truncation
	pd.set_option('display.max_rows', 50)         # Show up to 50 rows
	pd.set_option('display.max_columns', None)    # Show all columns
	pd.set_option('display.width', 1000)          # Widen the output area
	pd.set_option('display.max_colwidth', 150)    # Show more text in the 'text' column

	# Sort by the anomaly score to see the most anomalous first and show the top 20
	print(anomalies.sort_values(by='ae_reconstruction_error_zscore', ascending=False).head(20))

else:
	print("\nNo anomalies detected above the specified threshold.")

	# --- 8. Investigating the Latent Space (Optional) ---
print("\n--- Investigating the Latent Space ---")

# First, get the encoded representation of the entire dataset
model.eval()
with torch.no_grad():
	# We only need the encoder part for this
	encoded_data_tensor = model.encoder(scaled_data_tensor.to(device))
	encoded_data_np = encoded_data_tensor.cpu().numpy()

# Create a DataFrame for the encoded dimensions
encoded_df = pd.DataFrame(encoded_data_np, columns=[f'latent_dim_{i+1}' for i in range(encoding_dim)])

# Combine with the original (unscaled) features to make interpretation easier
combined_df = pd.concat([df[features].reset_index(drop=True), encoded_df], axis=1)

# Calculate the correlation matrix
correlation_matrix = combined_df.corr()

# Get the correlation of each latent dimension with the original features
latent_correlations = correlation_matrix[encoded_df.columns].loc[features]

print("\nCorrelation of Latent Dimensions with Original Features:")
# Display with a heatmap-style background for easier reading
print(latent_correlations)
print("\nStandard Deviation of each Latent Dimension:")
print(encoded_df.std())

"Successfully loaded 'final_dataset.parquet'. Shape: (26622, 36)"

Number of rows (lines): 26622


'First 5 rows of the dataset:'

,user_id,name_review_user,time,rating,text,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,price,state,category_main,cat_American restaurant,cat_Coffee shop,cat_Department store,cat_Fast food restaurant,cat_Grocery store,cat_Hotel,cat_Mexican restaurant,cat_Other,cat_Pizza restaurant,cat_Restaurant,cat_Shopping mall,price_encoded,year,month,weekday,hour,text_len,log_num_reviews,rating_deviation,sentiment_polarity,sentiment_subjectivity,excessive_exclaim
0,107813727678662505612,Felicia Wall,1575939675467,5,Coffee is great and so is the service,0x56c67c7da439c267:0x72f12aacd98f7f2e,60.561418,-151.285291,[Cafe],4.6,72,None,Open ⋅ Closes 3PM,Cafe,False,False,False,False,False,False,False,True,False,False,False,0,2019,12,1,1,37,4.290459,0.4,0.800000,0.750000,False
1,103473424444871906765,Garry Hanley,1602702499311,5,Easy in and out experience. Reasonable pricing for the convenience. We use them for most all our DMV needs.,0x56c899de8ee5d2a5:0xb15f3c48a1dc94f0,61.137792,-149.836917,"[Department of motor vehicles, Auto tag agency, State government office]",4.2,114,None,Closed ⋅ Opens 9AM,Department of motor vehicles,False,False,False,False,False,False,False,True,False,False,False,0,2020,10,2,19,109,4.744932,0.8,0.377778,0.644444,False
2,104393976961042472712,Nathan Harrison,1559438084515,4,"Great lunch, service could be better as there wasn't many people there.",0x56c894263c293e31:0x39aa7cb841273597,61.228926,-149.741000,"[Restaurant, Brewpub, Pizza restaurant, Sports bar]",3.9,648,$$,None,Restaurant,False,False,False,False,False,False,False,False,False,True,False,2,2019,6,6,1,71,6.475433,0.1,0.600000,0.583333,False
3,110829207026204377440,Terry Oakes,1578766259352,3,Small portions and do not give you what the menu lists.,0x51325b0542590ed1:0xaba794ec633521ef,64.821774,-147.713617,[Restaurant],4.2,125,None,Closed ⋅ Opens 11AM Sun,Restaurant,False,False,False,False,False,False,False,False,False,True,False,0,2020,1,5,18,55,4.836282,-1.2,-0.250000,0.400000,False
4,108158723955787868825,Lee Petersen,1501725448031,4,"Nice gym, what I expect from a Planet Fitness. Bathrooms and locker rooms are clean. Good variety of machines. Sometimes a bit crowded, but not to...",0x51325acaa661fa4d:0xa0cf5a8e5d34cbde,64.835650,-147.742050,"[Gym, Fitness center, Personal trainer, Spa and health club, Tanning salon]",4.1,208,None,Open 24 hours,Gym,False,False,False,False,False,False,False,True,False,False,False,0,2017,8,3,1,152,5.342334,-0.1,0.241667,0.741667,False


'\nAutoencoder Model Summary (PyTorch):'

...

'\nReconstruction errors calculated and z-scored.'

'First 10 reviews with their z-scored autoencoder reconstruction error:'

,ae_reconstruction_error_zscore,rating,text_len,rating_deviation,sentiment_polarity,sentiment_subjectivity,excessive_exclaim,avg_rating,num_of_reviews,log_num_reviews,price_encoded,year,month,weekday,hour,cat_American restaurant,cat_Coffee shop,cat_Department store,cat_Fast food restaurant,cat_Grocery store,cat_Hotel,cat_Mexican restaurant,cat_Other,cat_Pizza restaurant,cat_Restaurant,cat_Shopping mall
0,-0.319569,5,37,0.4,0.800000,0.750000,False,4.6,72,4.290459,0,2019,12,1,1,False,False,False,False,False,False,False,True,False,False,False
1,-0.198078,5,109,0.8,0.377778,0.644444,False,4.2,114,4.744932,0,2020,10,2,19,False,False,False,False,False,False,False,True,False,False,False
2,-0.193262,4,71,0.1,0.600000,0.583333,False,3.9,648,6.475433,2,2019,6,6,1,False,False,False,False,False,False,False,False,False,True,False
3,-0.012411,3,55,-1.2,-0.250000,0.400000,False,4.2,125,4.836282,0,2020,1,5,18,False,False,False,False,False,False,False,False,False,True,False
4,-0.274464,4,152,-0.1,0.241667,0.741667,False,4.1,208,5.342334,0,2017,8,3,1,False,False,False,False,False,False,False,True,False,False,False
5,-0.123624,5,123,0.8,0.250000,0.812500,False,4.2,328,5.796058,2,2019,7,2,3,False,False,False,False,False,False,True,False,False,False,False
6,0.727963,1,64,-2.7,-1.000000,0.900000,False,3.7,718,6.577861,1,2019,6,4,6,False,False,False,True,False,False,False,False,False,False,False
7,0.153349,5,27,0.3,1.000000,0.300000,False,4.7,412,6.023448,0,2019,1,0,4,False,False,False,False,False,False,False,True,False,False,False
8,0.096131,1,137,-3.5,0.300000,0.300000,False,4.5,138,4.934474,0,2019,5,6,20,False,False,False,False,False,False,False,True,False,False,False
9,0.112083,5,432,0.3,0.137500,0.617361,False,4.7,547,6.306275,2,2017,3,0,4,False,False,False,False,False,False,False,True,False,False,False


'\nNumber of potential anomalies (z-score > 2.0): 699'


Displaying the top 20 anomalies with all their features (sorted by anomaly score):
                     user_id    name_review_user           time  rating                                                                                                                                                   text                                gmap_id   latitude   longitude                                                                                                                                               category  avg_rating  num_of_reviews price                                  state            category_main  cat_American restaurant  cat_Coffee shop  cat_Department store  cat_Fast food restaurant  cat_Grocery store  cat_Hotel  cat_Mexican restaurant  cat_Other  cat_Pizza restaurant  cat_Restaurant  cat_Shopping mall  price_encoded  year  month  weekday  hour  text_len  log_num_reviews  rating_deviation  sentiment_polarity  sentiment_subjectivity  excessive_exclaim  ae_reconstruction_